In [4]:
import numpy as np
from scipy.special import factorial, comb

def compute_a(theta1, theta21, theta22, theta3, k0):
    a = np.zeros(k0+1)
    a[0] = 1
    for k in range(1, k0+1):
        a[k] = -theta21 * a[k-1] - theta22 * theta3 * a[k-2]
    return a

def compute_s(a, theta1, theta21, theta22, theta3, k0):
    m_vals = np.arange(k0 + 1)
    s = np.zeros((k0+1, 2, 2))
    for m in m_vals:
        k_vals = np.arange(m+1)
        l_vals = m - k_vals
        mat11 = np.sum(theta1**2 * comb(m, k_vals) * a[k_vals] * a[l_vals] / factorial(m+1))
        mat12 = np.sum(theta1**2 * theta3 * comb(m+1, k_vals) * a[k_vals] * a[l_vals] / factorial(m+2))
        mat22 = np.sum(theta3**2 * theta1**2 * comb(m+2, k_vals) * a[k_vals] * a[l_vals] / factorial(m+3))
        s[m] = np.array([[mat11, mat12], [mat12, mat22]])
    return s

def calculate_d_e(s, k0):
    d = {k: np.zeros_like(s[0]) for k in range(k0+1)}
    e = {k: 0 for k in range(k0+1)}
    d[0] = np.linalg.inv(s[0])
    e[0] = np.log(np.linalg.det(s[0]))
    for m in range(1, k0+1):
        for p in range(m):
            for q in range(m):
                if p + q > m-1:
                    continue
                r = m - p - q - 1
                # 修正: 'ij,jk,kl->il' を 'ij,jkl,kl->il' に変更して、s[q+1] の次元を合わせます。
                d[m] += factorial(m-1)/(factorial(p)*factorial(q)*factorial(r)) * np.einsum('ij,jkl,kl->il', d[p], s[q+1], d[r])
        e[m] = np.log(np.linalg.det(s[0]))
        for p in range(1, m+1):
            q = m - p
            # 修正: np.dot の代わりに np.einsum を使用して次元を合わせます。
            e[m] += - factorial(m)/factorial(p) * np.einsum('ij,ji->', d[p], s[q])
    return d, e


def D(a, x, y, x_, y_, theta1, theta21, theta22, theta3, k0, h):
    k_vals = np.arange(1, k0+1)
    f = np.sum((a[k_vals] * x_ - theta22 * a[k_vals-1] * y_) * h**k_vals / factorial(k_vals))
    g = np.sum(theta3 * (a[k_vals-1] * x_ - theta22 * a[k_vals-2] * y_) * h**k_vals / factorial(k_vals))
    Dx = h**(-1/2) * (x - x_ - f)
    Dy = h**(-3/2) * (y - y_ - g)
    return np.array([Dx, Dy])

def QL(X, Y, theta1, theta21, theta22, theta3, k0, h):
    a = compute_a(theta1, theta21, theta22, theta3, k0)
    s = compute_s(a, theta1, theta21, theta22, theta3, k0)
    d, e = calculate_d_e(s, k0)
    Q = 0
    for i in range(1, len(X)):
        D_ = D(a, X[i], Y[i], X[i-1], Y[i-1], theta1, theta21, theta22, theta3, k0, h)
        Q += np.sum(-h**np.arange(k0+1) * (np.trace(d @ np.outer(D_, D_)) + e))
    return Q


In [5]:
QL(np.array([0, 1]), np.array([0, 1]), 1, 1, 1, 1, 1, 1e-3)

/var/folders/db/b1td2bb97db6ppr5q5fk5_xc0000gn/T/ipykernel_98685/298712388.py:27: RuntimeWarning: invalid value encountered in log
  e[0] = np.log(np.linalg.det(s[0]))


ValueError: einstein sum subscripts string contains too many subscripts for operand 1